In [ ]:
'''
[문제] 픽사베이에서 '구슬'을 검색하여, 검색된 모든 페이지의 이미지 다운로드
- 다운로드 폴더 : 구슬
- 파일명 : 1_1.jpg
'''

In [11]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os
import requests

# 크롬 드라이버 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# 웹 브라우저 실행
driver = webdriver.Chrome(options=options)

# 픽사베이 접속
url = 'https://pixabay.com/ko/'
driver.get(url)

sleep(2)

search_word = '청개구리'

# 검색창에 검색어 입력
search = driver.find_element(By.CSS_SELECTOR, 'form > input')
search.send_keys(search_word)
search.send_keys(Keys.ENTER)

# 총 페이지 수 
total_page = driver.find_element(By.XPATH, '//*[@id="content"]//form')
total_page_count = int(total_page.text.replace('/',''))

# 폴더 생성
os.makedirs(search_word, exist_ok=True)

# 이미지 엘리먼트 추출
for current_page in range(1, int(total_page_count)+1):
    print(f'{current_page} 페이지 작업 시작')
    img_elements = driver.find_elements(By.CSS_SELECTOR, 'a > img[alt]')
    
    for idx, img in enumerate(img_elements, 1):
        if img.get_attribute('data-lazy'):
            src = img.get_attribute('data-lazy')
        else:
            src = img.get_attribute('src')

        # 주소로 이미지 파일 요청
        img_response = requests.get(src).content
    
        # 파일 저장
        file_path = f'{search_word}/{current_page}_{idx}.jpg'
        with open(file_path, 'wb') as file:
            file.write(img_response)
            
    # 마지막 페이지 이후 반복문 탈출
    if current_page == total_page_count:
        break
    
    # 다음 페이지 클릭
    if current_page >=2:
        next_page_button = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[1]/div/div[1]/div/a[2]')
    else:
        next_page_button = driver.find_element(By.XPATH, '//*[@id="content"]/div/div[1]/div/div[1]/div/a')
        
    next_page_button.send_keys(Keys.RETURN)
    
    sleep(3)
    
print('==작업 완료==')